#### Modify the config

In [8]:
from mmcv import Config
root_path = '/home/zhsong/Code/'
cfg = Config.fromfile(root_path+'mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py')

#### Data visdrone2019

In [9]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
root_path = '/home/zhsong/Code/'
classes = ('pedestrian', 'people','bicycle', 'car', 'van','truck','tricycle','awning-tricycle','bus', 'motor')

cfg.dataset_type = 'CocoDataset'
cfg.data_root = root_path + 'datasets/VisDrone2019-DET/'

cfg.data.test.type = 'CocoDataset'
cfg.data.test.data_root = cfg.data_root
cfg.data.test.ann_file = cfg.data_root + 'annotations/VisDrone2019-DET-test-dev-coco.json'
cfg.data.test.img_prefix = cfg.data_root + 'VisDrone2019-DET-test-dev/images/'
cfg.data.test.classes = classes

# cfg.train_dataset.dataset.data_root = cfg.data_root
cfg.train_dataset.dataset.ann_file = cfg.data_root + 'annotations/VisDrone2019-DET-train-coco.json'
cfg.train_dataset.dataset.img_prefix =  cfg.data_root + 'VisDrone2019-DET-train/images/'
cfg.train_dataset.dataset.classes = classes
cfg.data.train=cfg.train_dataset

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = cfg.data_root
cfg.data.val.ann_file = cfg.data_root + 'annotations/VisDrone2019-DET-val-coco.json'
cfg.data.val.img_prefix =  cfg.data_root + 'VisDrone2019-DET-val/images/'
cfg.data.val.classes = classes
# modify num classes of the model in box head
cfg.model.bbox_head.num_classes = 10
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = root_path + 'mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'

# Set up working dir to save files and logs.
cfg.work_dir = root_path + 'mmdetection/exps_out/yolox_visdrone'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.max_epochs = 1
cfg.evaluation.metric = 'bbox'
cfg.evaluation.classwise = True
# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
optimizer = dict(
    type='SGD',
    lr=0.0025,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup=None,
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=300)
checkpoint_config = dict(interval=1)
log_config = dict(interval=10, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,
        priority=49)
]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = '/home/zhsong/Code/mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_095

#### Train yolox

Finally, lets initialize the dataset and detector, then train a new detector!

In [10]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=2.15s)
creating index...
index created!
loading annotations into memory...


2022-03-09 20:24:25,919 - mmdet - INFO - load checkpoint from local path: /home/zhsong/Code/mmdetection/checkpoints/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
2022-03-09 20:24:25,983 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 128, 1, 1]) from checkpoint, the shape in current model is torch.Size([10, 128, 1, 1]).
size mismatch for bbox_head.multi_level_conv_cls.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([10]).
size mismatch for bbox_head.multi_level_conv_cls.1.weight: copying a param with shape torch.Size([80, 128, 1, 1]) from checkpoint, the shape in current model is torch.Size([10, 128, 1, 1]).
size mismatch for bbox_head.multi_level_conv_cls.1.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([10]).
size mismatch for bbox_

Done (t=1.11s)
creating index...
index created!


/home/zhsong/anaconda3/envs/UAVs/lib/python3.9/site-packages/mmcv/runner/hooks/evaluation.py:222: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')
2022-03-09 20:24:34,312 - mmdet - INFO - Epoch [1][10/809]	lr: 2.498e-03, eta: 2 days, 8:01:07, time: 0.831, data_time: 0.405, memory: 7495, loss_cls: 2.3967, loss_bbox: 3.0278, loss_obj: 3.1244, loss: 8.5489
2022-03-09 20:24:38,538 - mmdet - INFO - Epoch [1][20/809]	lr: 2.498e-03, eta: 1 day, 18:15:08, time: 0.423, data_time: 0.018, memory: 7495, loss_cls: 1.4782, loss_bbox: 2.9743, loss_obj: 2.4809, loss: 6.9334
2022-03-09 20:24:42,713 - mmdet - INFO - Epoch [1][30/809]	lr: 2.498e-03, eta: 1 day, 13:32:48, time: 0.417, data_time: 0.018, memory: 7495, loss_cls: 1.4295, loss_bbox: 2.9350, loss_obj: 2.3871, loss: 6.7516
2022-03-09 20:24:45,995 - mmdet - INFO - Epoch [1][40/809]	lr: 2.498e-03, eta: 1 day, 9:41:12, time: 0.328, data_time: 0.024, memory: 7495, loss_cls: 1.344

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 548/548, 54.8 task/s, elapsed: 10s, ETA:     0s

...

Loading and preparing results...
DONE (t=1.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=108.42s).
Accumulating evaluation results...


2022-03-09 20:32:04,422 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.069
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.134
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.243
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.123
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.126
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.127
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.057
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.219
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=3.79s).


2022-03-09 20:32:04,834 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_1.pth.
2022-03-09 20:32:04,835 - mmdet - INFO - Best bbox_mAP is 0.0690 at 1 epoch.
2022-03-09 20:32:04,836 - mmdet - INFO - Epoch(val) [1][548]	bbox_mAP: 0.0690, bbox_mAP_50: 0.1340, bbox_mAP_75: 0.0670, bbox_mAP_s: 0.0250, bbox_mAP_m: 0.1230, bbox_mAP_l: 0.2430, bbox_mAP_copypaste: 0.069 0.134 0.067 0.025 0.123 0.243
2022-03-09 20:32:12,757 - mmdet - INFO - Epoch [2][10/809]	lr: 2.499e-03, eta: 1 day, 3:29:11, time: 0.791, data_time: 0.371, memory: 7495, loss_cls: 1.1072, loss_bbox: 2.6996, loss_obj: 1.8753, loss: 5.6822
2022-03-09 20:32:16,643 - mmdet - INFO - Epoch [2][20/809]	lr: 2.499e-03, eta: 1 day, 3:28:08, time: 0.389, data_time: 0.018, memory: 7495, loss_cls: 1.1039, loss_bbox: 2.8227, loss_obj: 1.9121, loss: 5.8387
2022-03-09 20:32:21,059 - mmdet - INFO - Epoch [2][30/809]	lr: 2.499e-03, eta: 1 day, 3:29:38, time: 0.442, data_time: 0.016, memory: 7495, loss_cls: 1.1144, loss_bbox: 2

KeyboardInterrupt: 

python tools/train.py configs/yolox/yolox_s_8x8_300e_coco.py

In [53]:
pip install -U ipykernel

     |████████████████████████████████| 128 kB 603 kB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 3.5 MB/s eta 0:00:01
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.5
    Uninstalling ipykernel-5.5.5:
      Successfully uninstalled ipykernel-5.5.5
Note: you may need to restart the kernel to use updated packages.
